In [ ]:
#Imports
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import xarray as xr
import folium
import numpy as np
import pandas as pd
import datetime as dt

from eodag import EODataAccessGateway
from eodag import setup_logging

from rasterio.crs import CRS
from rioxarray.merge import merge_arrays

from dotenv import dotenv_values

# Setup Verbose Values:
# 0: no logging and no progress bar
# 1: no logging but progress bars displayed
# 2: log at the INFO level
# 3: log at the DEBUG level (even more information)

setup_logging(verbose=0)


# EODAG - merging

EODAG (Earth Observation Data Access Gateway) is a command line tool and a Python package for searching and downloading remotely sensed images while offering a unified API for data access regardless of the data provider.

EODAG gives you an easy way to access products from more than 10 providers, with more than 50 different product types (Sentinel 1, Sentinel 2, Sentinel 3, Landsat, etc.) that can be searched and downloaded.

In [ ]:
#Get Secrets from .env File
secrets = dotenv_values('.env')

In [ ]:
#Create Folders for saving Data, serializing and post processing.

# Path where the Data should be stored ('c:\\Users\\theUSER\\eodag-data')
root = '../eodag-data/'

workspace_download = os.path.join(root,'eodag_workspace_download')
workspace_serialize = os.path.join(root,'eodag_workspace_serialize_deserialize')
workspace_post_process = os.path.join(root,'eodag_workspace_post_process')
workspaces = [workspace_download, workspace_serialize, workspace_post_process]

for ws in workspaces:
    ws = os.path.abspath(ws)
    
    if not os.path.isdir(ws):
        os.mkdir(ws)
        print(f'Created Folder: {ws}')
    else:
        print(f'Folder already exists: {ws}')

## Data Acquisition
The Lines of Code in this chapter, which essentialy load the data, are not further explained here, since the Notebooks 01 and 02 do that.

In [ ]:
# 1. Configure
#Create EODAG Object and set preferred Provider
dag = EODataAccessGateway()
dag.set_preferred_provider("cop_dataspace") # Copernicus Data Space Ecosystem

dag.update_providers_config(f"""
    cop_dataspace:
        download:
            outputs_prefix: {os.path.abspath(workspace_download)}
        auth:
            credentials:
                username: {secrets['USER_KEY']}
                password: {secrets['USER_SECRET']}
""")


In [ ]:
# Deserialize the Search Results
output_file = os.path.join(workspace_serialize, "search_results.geojson")
deserialized_search_results = dag.deserialize_and_register(output_file)

print(f"Got {len(deserialized_search_results)} deserialized products.")

In [ ]:
# Download Single Product
product = deserialized_search_results[2]
path = dag.download(product)

# Download multiple Products
products = deserialized_search_results[1:3]
paths = dag.download_all(products)


In [ ]:
# Set Boundingbox for Area inside the Tile.
latmin, latmax = 48.1, 48.35
lonmin, lonmax = 16.1, 16.6
extent = {'lonmin': lonmin, 'latmin': latmin, 'lonmax': lonmax, 'latmax': latmax}

# Folium Map
fmap = folium.Map(location=(np.array([latmin, latmax]).mean(), np.array([lonmin, lonmax]).mean()), zoom_start=9)
folium.Rectangle(bounds=[[latmin, lonmin],[latmax, lonmax]], color="red").add_to(fmap)
folium.GeoJson(
    data=deserialized_search_results[:],  # SearchResult has a __geo_interface__ interface used by folium to get its GeoJSON representation, single results dont work (this [2:3] instead of [2])
    tooltip=folium.GeoJsonTooltip(fields=["title"])
).add_to(fmap)
fmap

In [ ]:
# Setting common Parameters for all further image processing
common_params = dict(
    crs=CRS.from_epsg(4326),               # the downloaded images are in 4326, don't reproject them
    resolution=0.0006,                     # but lower their resolution (0.0006 should be 60m in 100km)
    extent=(lonmin,latmin,lonmax,latmax)   # and zoom over/crop the area of interest
)

# Define basic Functions for future operations
def normalized_difference(a, b):
    return (a - b*1.)/(a + b)

def normalize(a):
    return (a - a.min())/(a.max() - a.min())
    

## Post Process - Merging

In [ ]:
# Get a list of all available Bands (assets)
def get_assets(root:str, res=60):
    jp2_files = [file for dirs in os.walk(root, topdown=True)
                     for file in dirs[2] if file.endswith(f"_{res}m.jp2")]
    assets = [file.split('_')[2] for file in jp2_files if file.startswith('T')]
    return assets
    
    
assets = get_assets(path, res=10)
assets

In [ ]:
# Functions for loading data into datasets.

def load_single_product(product, bands:list):
    loaded_data = {}

    for band in bands:
        data = product.get_data(band=band, **common_params)
        data = data.squeeze()

        time_str = product.properties['startTimeFromAscendingNode']
        date = dt.datetime.strptime(time_str,'%Y-%m-%dT%H:%M:%S.%f%z')

        data = data.expand_dims(dim={'time':[date.date()]})
        data.name = band
        loaded_data[band] = data
    ds = xr.Dataset(loaded_data)
    return ds

def load_multiple_timestamps(products, bands:list):
    loaded_data = {}
    # Combine multiple Dataarrays into one Dataset
    for band in bands:
        single_band_ts = []
        # Load one Band of each product and concat them into an xarray Dataarray
        for prod in products: 
            data = prod.get_data(band=band, **common_params)
            data = data.squeeze()

            time_str = prod.properties['startTimeFromAscendingNode'][:-1] #Remove Timezone info
            date = pd.to_datetime([time_str])

            data = data.expand_dims(time=date)
            data.name = band
            single_band_ts.append(data)
            
        ts =  xr.concat(single_band_ts, dim='time')
        loaded_data[band] = ts   

    ds = xr.Dataset(loaded_data)
    return ds

In [ ]:
# Loading multiple Bands into a dataset
ds1 = load_single_product(product=product, bands=['B04', 'B03', 'B02', 'B08', 'TCI'])

In [ ]:
# Loading multiple Bands into a dataset with multiple Timestamps
ds = load_multiple_timestamps(products=products, bands=['B04', 'B03', 'B02', 'B08', 'TCI'])

#### Merging of Arrays

In this example two geographicaly overlapping images are merged, and the NDVI is calculated. Note that only Dataarrays are merged but not Datasets.

In [ ]:
# Load Data of Bands 4 and 8. The Images are from the same time, but different tiles.
RED_1 = deserialized_search_results[2].get_data(band="B04", **common_params)
RED_2 = deserialized_search_results[3].get_data(band="B04", **common_params)

NIR_1 = deserialized_search_results[2].get_data(band="B08", **common_params)
NIR_2 = deserialized_search_results[3].get_data(band="B08", **common_params)

In [ ]:
# Using merge_arrays returns the geospatially merged data.
merged_red = merge_arrays([RED_1, RED_2], method='max') # methods = ['first', 'last', 'min', 'max', 'sum', 'count']
merged_nir = merge_arrays([NIR_1, NIR_2], method='max')

merged_ndvi = normalized_difference(merged_nir, merged_red)

In [ ]:
merged_ndvi.plot(cmap='RdYlGn', center=False, size=6, aspect='auto')